# EDA

In [1]:
# check that the notebook is using the correct virtual environment
# if you dont call it `umads_venv`, change the assertion

import sys
venv = sys.executable.split('/')[-3]
assert venv == 'umads_venv'

In [2]:
import pandas as pd
import os
import altair as alt

# NY Arrest Data Set

In [ ]:
crime_2017_df = pd.read_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2017.csv'))
crime_2018_df = pd.read_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2018.csv'))
crime_2019_df = pd.read_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2019.csv'))

print(f'2017 crime df shape: {crime_2017_df.shape}')
print(f'2018 crime df shape: {crime_2018_df.shape}')
print(f'2019 crime df shape: {crime_2019_df.shape}')

In [ ]:
crime_2017_df.head()

In [ ]:
# format date string

crime_2017_df['ARREST_DATE_STR'] = crime_2017_df['ARREST_DATE'].copy()
crime_2017_df['ARREST_DATE'] = pd.to_datetime(crime_2017_df['ARREST_DATE_STR']).copy()
crime_2017_df['ARREST_DATE_WEEK'] = crime_2017_df['ARREST_DATE'].dt.isocalendar().week
crime_2017_df.info()

In [ ]:
# nothing interesting here
crime_2017_df.describe()

### High Level Numbers

In [ ]:
print(f'Crimes in 2017: {crime_2017_df.shape[0]:,}')
print(f'Total Zip Codes in Data Set: {len(crime_2017_df.zip_code.unique())}')
print(f'Unique Crime Types: {len(crime_2017_df.PD_CD.unique())}')
print(f'')


### Crimes by Zip Code 

In [ ]:
crime_by_zip = crime_2017_df.groupby(['zip_code'], as_index=False).agg({'ARREST_KEY': 'count'}).sort_values(by='ARREST_KEY', ascending=False)
crime_by_zip['Average_Arrests'] = crime_by_zip.ARREST_KEY.mean()
crime_by_zip

In [ ]:
# anything under the line has less than average crime, anything over has more

alt.Chart(crime_by_zip).mark_bar().encode(
    x=alt.X('zip_code:O', sort=alt.EncodingSortField('y', order='descending')),
    y=alt.Y('ARREST_KEY'),
    y2=alt.Y2('Average_Arrests'))

In [ ]:
# fewest arrests of the year on Christmas, makes sense

crime_by_day = crime_2017_df.groupby(['ARREST_DATE'], as_index=False).agg({'ARREST_KEY': 'count'}).sort_values(by='ARREST_KEY', ascending=False)
crime_by_day

In [ ]:
# arrests usually lowest on weekends
# arrest decrase throughout the week

alt.Chart(crime_by_day).mark_bar().encode(
    x=alt.X('ARREST_DATE:O'),
    y=alt.Y('ARREST_KEY'))

In [ ]:
# most to least common crimes

crime_2017_df.PD_DESC.value_counts()

In [ ]:
# crime by week by zip
# randomly split zip codes into 2 groups to keep data small enough for altair

week_zip_crime = crime_2017_df.groupby(['zip_code', 'ARREST_DATE_WEEK'], as_index=False).agg({'ARREST_KEY': 'count'})
temp1 = week_zip_crime[week_zip_crime.zip_code <= 11218].copy()
temp2 = week_zip_crime[week_zip_crime.zip_code > 11218].copy()

In [ ]:
alt.Chart(temp1).mark_rect().encode(
    x='zip_code:O',
    y='ARREST_DATE_WEEK:O',
    color='ARREST_KEY:Q'
).properties(width=1200, height=700)

In [ ]:
alt.Chart(temp2).mark_rect().encode(
    x='zip_code:O',
    y='ARREST_DATE_WEEK:O',
    color='ARREST_KEY:Q'
).properties(width=1200, height=700)

## Crimes by Demographic

In [ ]:
crime_by_age_group = crime_2017_df.groupby(['AGE_GROUP'], as_index=False).agg({'ARREST_KEY': 'count'})

In [ ]:
alt.Chart(crime_by_age_group).mark_bar().encode(
    x=alt.X('AGE_GROUP:O', sort=alt.EncodingSortField('y', order='descending')),
    y=alt.Y('ARREST_KEY'))

In [ ]:
crime_by_age_sex = crime_2017_df.groupby(['PERP_SEX'], as_index=False).agg({'ARREST_KEY': 'count'})

In [ ]:
alt.Chart(crime_by_age_sex).mark_bar().encode(
    x=alt.X('PERP_SEX:O', sort=alt.EncodingSortField('y', order='descending')),
    y=alt.Y('ARREST_KEY'))

In [ ]:
crime_by_age_race = crime_2017_df.groupby(['PERP_RACE'], as_index=False).agg({'ARREST_KEY': 'count'})

In [ ]:
alt.Chart(crime_by_age_race).mark_bar().encode(
    x=alt.X('PERP_RACE:O', sort=alt.EncodingSortField('y', order='descending')),
    y=alt.Y('ARREST_KEY'))

Site: https://www.zillow.com/research/data/
    
HOME VALUES
Zillow Home Value Index (ZHVI)
by zip, filter down to State=NY, city=NY

In [88]:
home_value_df = pd.read_csv(os.path.join('..','data','cleaned_data','ny_zhvi.csv'))
home_value_df.head()

,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/96,2/29/96,3/31/96,...,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21,2/28/21,3/31/21
0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,227153.0,225738.0,225814.0,...,1091641,1087154,1085121,1086397,1082450,1078831,1074640,1066881,1069011,1073214
1,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,200946.0,200989.0,201100.0,...,998619,998643,1001866,1013305,1016169,1014689,1009092,1011561,1021442,1029770
2,10002,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,NaN,NaN,...,954773,944821,930869,917405,910523,905972,897203,884429,877971,878429
3,11226,Zip,NY,NY,New York,New York-Newark-Jersey City,Kings County,178939.0,178042.0,176987.0,...,657204,658504,661081,659057,657760,654971,654540,647604,638926,626094
4,10467,Zip,NY,NY,New York,New York-Newark-Jersey City,Bronx County,69718.0,69317.0,69163.0,...,205185,204679,204260,201885,200275,199158,199852,201772,204815,209057


In [89]:
home_value_df.drop(['RegionType','StateName','State','City','Metro','CountyName'], inplace=True, axis=1) 
home_value_df.rename({'RegionName': 'zipcode'}, inplace=True, axis=1)
home_value_df.set_index('zipcode',inplace=True)
home_value_df.head()

,1/31/96,2/29/96,3/31/96,4/30/96,5/31/96,6/30/96,7/31/96,8/31/96,9/30/96,10/31/96,...,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21,2/28/21,3/31/21
zipcode,,,,,,,,,,,,,,,,,,,,,
10025,227153.0,225738.0,225814.0,225790.0,226306.0,226934.0,226823.0,227619.0,227965.0,229548.0,...,1091641,1087154,1085121,1086397,1082450,1078831,1074640,1066881,1069011,1073214
10023,200946.0,200989.0,201100.0,201816.0,202857.0,203710.0,203994.0,204078.0,204399.0,205007.0,...,998619,998643,1001866,1013305,1016169,1014689,1009092,1011561,1021442,1029770
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,954773,944821,930869,917405,910523,905972,897203,884429,877971,878429
11226,178939.0,178042.0,176987.0,175656.0,175035.0,175304.0,175524.0,175670.0,175873.0,176143.0,...,657204,658504,661081,659057,657760,654971,654540,647604,638926,626094
10467,69718.0,69317.0,69163.0,68621.0,68456.0,68262.0,68313.0,68104.0,67999.0,68069.0,...,205185,204679,204260,201885,200275,199158,199852,201772,204815,209057


In [90]:
home_value_df_transposed = home_value_df.T
home_value_df_transposed.reset_index(inplace=True)
cols = [str(a) for a in list(home_value_df_transposed)]
cols[0] = 'date_str'
home_value_df_transposed.columns = cols
home_value_df_transposed.head(20)

,date_str,10025,10023,10002,11226,10467,10016,11201,11235,11375,...,10282,11363,10006,10004,10464,11430,11243,13784,10020,10118
0,1/31/96,227153.0,200946.0,NaN,178939.0,69718.0,174336.0,156669.0,95560.0,123124.0,...,NaN,309452.0,NaN,248809.0,256282.0,230689.0,286139.0,46407.0,NaN,NaN
1,2/29/96,225738.0,200989.0,NaN,178042.0,69317.0,173744.0,155498.0,95372.0,122453.0,...,NaN,309853.0,NaN,250489.0,255600.0,229823.0,287071.0,46173.0,NaN,NaN
2,3/31/96,225814.0,201100.0,NaN,176987.0,69163.0,173278.0,154861.0,95201.0,122106.0,...,NaN,310628.0,NaN,251016.0,255553.0,229441.0,285752.0,45971.0,NaN,NaN
3,4/30/96,225790.0,201816.0,NaN,175656.0,68621.0,172562.0,154515.0,95192.0,121411.0,...,NaN,311505.0,NaN,253376.0,255824.0,229070.0,284743.0,45346.0,NaN,NaN
4,5/31/96,226306.0,202857.0,NaN,175035.0,68456.0,172142.0,154860.0,95443.0,121131.0,...,NaN,312621.0,NaN,254340.0,256408.0,229226.0,281906.0,44720.0,NaN,NaN
5,6/30/96,226934.0,203710.0,NaN,175304.0,68262.0,171903.0,155328.0,95716.0,120852.0,...,NaN,313239.0,NaN,254404.0,256954.0,229358.0,280034.0,44154.0,NaN,NaN
6,7/31/96,226823.0,203994.0,NaN,175524.0,68313.0,171476.0,155535.0,96125.0,120842.0,...,NaN,314154.0,NaN,251976.0,257535.0,229106.0,277663.0,44048.0,NaN,NaN
7,8/31/96,227619.0,204078.0,NaN,175670.0,68104.0,171062.0,156142.0,96603.0,120776.0,...,NaN,314790.0,NaN,247483.0,257632.0,229191.0,274811.0,44198.0,NaN,NaN
8,9/30/96,227965.0,204399.0,NaN,175873.0,67999.0,170778.0,156631.0,97193.0,120955.0,...,NaN,314415.0,NaN,243493.0,258707.0,229391.0,272092.0,44356.0,NaN,NaN
9,10/31/96,229548.0,205007.0,NaN,176143.0,68069.0,170728.0,157629.0,97475.0,121157.0,...,NaN,314316.0,NaN,240438.0,260045.0,230089.0,268534.0,44315.0,NaN,NaN


In [91]:
alt.Chart(home_value_df_transposed[['date_str','10002']]).mark_line().encode(
    x=alt.X('date_str:T',sort=None),
    y=alt.Y('10002'))

alt.Chart(...)

In [92]:
# add year column
# group by year and zip, take mean home value
# combine into 1 df
# columns: date, zip, price

home_value_df_transposed_slim = pd.DataFrame(columns=['date_str', 'mean_zhvi',  'zipcode'])

for zip_ in list(home_value_df_transposed):
    if zip_ == 'date_str':
        continue
    current_zip = home_value_df_transposed[['date_str', zip_]].copy()
    current_zip['zipcode'] = zip_
    current_zip.columns = ['date_str', 'mean_zhvi',  'zipcode']
    home_value_df_transposed_slim = home_value_df_transposed_slim.append(current_zip)
home_value_df_transposed_slim['mean_zhvi'] = home_value_df_transposed_slim['mean_zhvi'].astype(float)

In [98]:
home_value_df_transposed_slim['date'] = pd.to_datetime(home_value_df_transposed_slim['date_str'])
home_value_df_transposed_slim['date_year'] = home_value_df_transposed_slim['date'].dt.isocalendar().year
zhvi_grouped = home_value_df_transposed_slim.groupby(['zipcode', 'date_year'], as_index=False).agg({'mean_zhvi':'mean'})
zhvi_grouped['zipcode_max'] = zhvi_grouped.groupby('zipcode')['mean_zhvi'].transform('max')
zhvi_grouped.head()

,zipcode,date_year,mean_zhvi,zipcode_max
0,10001,1996,167910.090909,1.003000e+06
1,10001,1997,177372.583333,1.003000e+06
2,10001,1998,211230.000000,1.003000e+06
3,10001,1999,245925.000000,1.003000e+06
4,10001,2000,291219.916667,1.003000e+06


In [100]:
alt.Chart(zhvi_grouped).mark_line().encode(
    x=alt.X('date_year',sort=None),
    y=alt.Y('mean_zhvi'),
    color='zipcode')

alt.Chart(...)

In [101]:
# remove the top valued zip codes

alt.Chart(zhvi_grouped[zhvi_grouped.zipcode_max < 1500000]).mark_line().encode(
    x=alt.X('date_year',sort=None),
    y=alt.Y('mean_zhvi'),
    color='zipcode')

alt.Chart(...)